In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

#AZURE OPEN AI MODEL INTEGRATION
from langchain.chat_models import init_chat_model

#OpenAI API Key unavailable, but Azure OpenAI keys available. Hence, using Azure OpenAI model.
model = init_chat_model(
    os.getenv("AZURE_OPENAI_LLM_MODEL"),
    model_provider="azure_openai",
    deployment_name=os.getenv("AZURE_OPENAI_LLM_MODEL"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2023-05-15"),
)

In [3]:
from langchain.tools import tool

@tool
def get_current_weather(location: str) -> str:
    """Get the current weather in a given location"""
    # For demonstration purposes, we'll return a static response.
    return f"The current weather in {location} is sunny with a temperature of 25°C."

@tool
def get_news_headlines() -> str:
    """Get the latest news headlines for a given topic"""
    # For demonstration purposes, we'll return a static response.
    return f"Latest news: 'AI is transforming the world', 'New advancements in renewable energy'."

model_with_tools = model.bind_tools([get_current_weather, get_news_headlines])

In [9]:
response = model_with_tools.invoke("What's the current weather in New York?")
for tool_call in response.tool_calls:
    print(f"Tool called: {tool_call['name']} with arguments {tool_call['args']}")
response

Tool called: get_current_weather with arguments {'location': 'New York'}


AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 90, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-chat-2025-10-03', 'system_fingerprint': 'fp_88bf7c189b', 'id': 'chatcmpl-CxUoZ7WRHuNT3Kcf1lgPJ9y3xLk7o', 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='lc_run--019bb6a5-32f4-72c3-aed5-470334bcb896-0', tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'New York'}, 'id': 'call_sfFQK0rAG8KSwiMIdHKodJDo', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 90, 'output_tokens': 17, 'total_tokens': 107, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0

In [10]:
#Tool execution loops

messages = [
    {"role": "user", "content": "What's the current weather in New York?"},
]

ai_response = model_with_tools.invoke(messages)

messages.append(ai_response)

for tool_call in ai_response.tool_calls:
    tool_result = get_current_weather.invoke(tool_call)
    messages.append(tool_result)

final_response = model_with_tools.invoke(messages)
print(final_response.text)

The current weather in New York is sunny with a temperature of 25°C.


In [15]:
def tool_execution_loop(model, user_query):
    messages = [
        {"role": "user", "content": user_query},
    ]

    while True:
        ai_response = model.invoke(messages)
        messages.append(ai_response)

        if not ai_response.tool_calls:
            break

        for tool_call in ai_response.tool_calls:
            if tool_call["name"] == "get_current_weather":
                tool_result = get_current_weather.invoke(tool_call)
            
            elif tool_call["name"] == "get_news_headlines":
                tool_result = get_news_headlines.invoke(tool_call)
            else:
                tool_result = f"Tool {tool_call['name']} not recognized."
            
            messages.append(tool_result)

    return ai_response.text

In [16]:
print(tool_execution_loop(model_with_tools, "What's the current weather in New York?"))
print(tool_execution_loop(model_with_tools, "Get me the latest news headlines."))

The current weather in New York is sunny with a temperature of 25°C.
Here are the latest news headlines:  
1. *AI is transforming the world*  
2. *New advancements in renewable energy*


In [17]:
# Process multiple queries in batch
queries = [
    "What's the current weather in New York?",
    "Get me the latest news headlines.",
    "What's the current weather in London?"
]

# Use batch processing
batch_results = []
for query in queries:
    result = tool_execution_loop(model_with_tools, query)
    batch_results.append({"query": query, "result": result})

# Display results
for item in batch_results:
    print(f"Query: {item['query']}")
    print(f"Result: {item['result']}\n")

Query: What's the current weather in New York?
Result: The current weather in New York is sunny with a temperature of 25°C.

Query: Get me the latest news headlines.
Result: Here are the latest news headlines:
1. "AI is transforming the world"  
2. "New advancements in renewable energy"

Query: What's the current weather in London?
Result: The current weather in London is sunny with a temperature of 25°C.

